#基础部分

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# from __future__ import print_function, division
import tensorflow as tf
from sklearn.utils import shuffle
import tifffile as tiff
from tensorflow import keras as K
import tensorflow.keras.layers as L
import numpy as np
import os
import time
import h5py
import argparse 
import random
import cv2
from tqdm import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LambdaCallback
# from keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from __future__ import print_function, division
from tensorflow.keras.layers import *
# from keras.layers import Dense,Dropout
from sklearn.utils import shuffle
# from keras.layers.core import Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
# from tensorflow.keras.layers.core import Lambda
from tensorflow.keras import backend as Kb
from collections import Counter
# from utils_1 import *

#模型

In [ ]:
def import_model(weightpath,train_para):
  ksize = 2 * r + 1
  inputshape = L.Input((ksize,ksize,hchn))
  
  output_feature = feature_extraction_CNN(inputshape,training_parameter=train_para, n_filters=64)
  logits = L.Dense(NUM_CLASS+1, activation = 'softmax')(output_feature)
  model = K.models.Model(inputshape, logits)
  adam = K.optimizers.Adam(lr=1e-3,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01)
  # adam = K.optimizers.Adam(lr=1e-6)
  model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics=['acc'])
  model.load_weights(weightpath)
  return model

In [ ]:
def tf_flatten(a):
  """Flatten tensor"""
  return tf.reshape(a, [-1])


def tf_repeat(a, repeats, axis=0):
  """TensorFlow version of np.repeat for 1D"""
  # https://github.com/tensorflow/tensorflow/issues/8521
  assert len(a.get_shape()) == 1

  a = tf.expand_dims(a, -1)
  a = tf.tile(a, [1, repeats])
  a = tf_flatten(a)
  return a


def tf_repeat_2d(a, repeats):
  """Tensorflow version of np.repeat for 2D"""

  assert len(a.get_shape()) == 2
  a = tf.expand_dims(a, 0)
  a = tf.tile(a, [repeats, 1, 1])
  return a


def tf_map_coordinates(input, coords, order=1):
  """Tensorflow verion of scipy.ndimage.map_coordinates

  Note that coords is transposed and only 2D is supported

  Parameters
  ----------
  input : tf.Tensor. shape = (s, s)
  coords : tf.Tensor. shape = (n_points, 2)
  """

  assert order == 1

  coords_lt = tf.cast(tf.floor(coords), 'int32')
  #coords_rb = tf.cast(tf.ceil(coords), 'int32')
  coords_rb = tf.cast(tf.math.ceil(coords), 'int32')
  coords_lb = tf.stack([coords_lt[:, 0], coords_rb[:, 1]], axis=1)
  coords_rt = tf.stack([coords_rb[:, 0], coords_lt[:, 1]], axis=1)

  vals_lt = tf.gather_nd(input, coords_lt)
  vals_rb = tf.gather_nd(input, coords_rb)
  vals_lb = tf.gather_nd(input, coords_lb)
  vals_rt = tf.gather_nd(input, coords_rt)

  coords_offset_lt = coords - tf.cast(coords_lt, 'float32')
  vals_t = vals_lt + (vals_rt - vals_lt) * coords_offset_lt[:, 0]
  vals_b = vals_lb + (vals_rb - vals_lb) * coords_offset_lt[:, 0]
  mapped_vals = vals_t + (vals_b - vals_t) * coords_offset_lt[:, 1]

  return mapped_vals


def sp_batch_map_coordinates(inputs, coords):
  """Reference implementation for batch_map_coordinates"""
  coords = coords.clip(0, inputs.shape[1] - 1)
  mapped_vals = np.array([
      sp_map_coordinates(input, coord.T, mode='nearest', order=1)
      for input, coord in zip(inputs, coords)
  ])
  return mapped_vals


def tf_batch_map_coordinates(input, coords, order=1):
  """Batch version of tf_map_coordinates

  Only supports 2D feature maps

  Parameters
  ----------
  input : tf.Tensor. shape = (b, s, s)
  coords : tf.Tensor. shape = (b, n_points, 2)

  Returns
  -------
  tf.Tensor. shape = (b, s, s)
  """

  input_shape = tf.shape(input)
  batch_size = input_shape[0]
  input_size = input_shape[1]
  n_coords = tf.shape(coords)[1]

  coords = tf.clip_by_value(coords, 0, tf.cast(input_size, 'float32') - 1)
  coords_lt = tf.cast(tf.floor(coords), 'int32')
  #coords_rb = tf.cast(tf.ceil(coords), 'int32')
  coords_rb = tf.cast(tf.math.ceil(coords), 'int32')
  coords_lb = tf.stack([coords_lt[..., 0], coords_rb[..., 1]], axis=-1)
  coords_rt = tf.stack([coords_rb[..., 0], coords_lt[..., 1]], axis=-1)

  idx = tf_repeat(tf.range(batch_size), n_coords)

  def _get_vals_by_coords(input, coords):
      indices = tf.stack([
          idx, tf_flatten(coords[..., 0]), tf_flatten(coords[..., 1])
      ], axis=-1)
      vals = tf.gather_nd(input, indices)
      vals = tf.reshape(vals, (batch_size, n_coords))
      return vals

  vals_lt = _get_vals_by_coords(input, coords_lt)
  vals_rb = _get_vals_by_coords(input, coords_rb)
  vals_lb = _get_vals_by_coords(input, coords_lb)
  vals_rt = _get_vals_by_coords(input, coords_rt)

  coords_offset_lt = coords - tf.cast(coords_lt, 'float32')
  vals_t = vals_lt + (vals_rt - vals_lt) * coords_offset_lt[..., 0]
  vals_b = vals_lb + (vals_rb - vals_lb) * coords_offset_lt[..., 0]
  mapped_vals = vals_t + (vals_b - vals_t) * coords_offset_lt[..., 1]

  return mapped_vals


def sp_batch_map_offsets(input, offsets):
  """Reference implementation for tf_batch_map_offsets"""

  batch_size = input.shape[0]
  input_size = input.shape[1]

  offsets = offsets.reshape(batch_size, -1, 2)
  grid = np.stack(np.mgrid[:input_size, :input_size], -1).reshape(-1, 2)
  grid = np.repeat([grid], batch_size, axis=0)
  coords = offsets + grid
  coords = coords.clip(0, input_size - 1)

  mapped_vals = sp_batch_map_coordinates(input, coords)
  return mapped_vals


def tf_batch_map_offsets(input, offsets, order=1):
  """Batch map offsets into input

  Parameters
  ---------
  input : tf.Tensor. shape = (b, s, s)
  offsets: tf.Tensor. shape = (b, s, s, 2)

  Returns
  -------
  tf.Tensor. shape = (b, s, s)
  """

  input_shape = tf.shape(input)
  batch_size = input_shape[0]
  input_size = input_shape[1]

  offsets = tf.reshape(offsets, (batch_size, -1, 2))
  grid = tf.meshgrid(
      tf.range(input_size), tf.range(input_size), indexing='ij'
  )
  grid = tf.stack(grid, axis=-1)
  grid = tf.cast(grid, 'float32')
  grid = tf.reshape(grid, (-1, 2))
  grid = tf_repeat_2d(grid, batch_size)
  coords = offsets + grid

  mapped_vals = tf_batch_map_coordinates(input, coords)
  return mapped_vals
def BN_LeakyReLU(input):
    
  norm = L.BatchNormalization(axis=-1)(input)
  output = L.LeakyReLU(alpha=0.2)(norm)
  
  return output
class ConvOffset2D(Conv2D):
  """ConvOffset2D

  Convolutional layer responsible for learning the 2D offsets and output the
  deformed feature map using bilinear interpolation

  Note that this layer does not perform convolution on the deformed feature
  map. See get_deform_cnn in cnn.py for usage
  """

  def __init__(self, filters, init_normal_stddev=0.01, **kwargs):
    """Init

    Parameters
    ----------
    filters : int
        Number of channel of the input feature map
    init_normal_stddev : float
        Normal kernel initialization
    **kwargs:
        Pass to superclass. See Con2D layer in Keras
    """

    self.filters = filters
    super(ConvOffset2D, self).__init__(
        self.filters * 2, (3, 3), padding='same', use_bias=False,
        kernel_initializer=K.initializers.RandomNormal(0, init_normal_stddev),
        **kwargs
    )

  def call(self, x):
    """Return the deformed featured map"""
    x_shape = x.get_shape()
    offsets = super(ConvOffset2D, self).call(x)

    # offsets: (b*c, h, w, 2)
    offsets = self._to_bc_h_w_2(offsets, x_shape)

    # x: (b*c, h, w)
    x = self._to_bc_h_w(x, x_shape)

    # X_offset: (b*c, h, w)
    x_offset = tf_batch_map_offsets(x, offsets)

    # x_offset: (b, h, w, c)
    x_offset = self._to_b_h_w_c(x_offset, x_shape)

    return x_offset

  def compute_output_shape(self, input_shape):
    """Output shape is the same as input shape

    Because this layer does only the deformation part
    """
    return input_shape

  @staticmethod
  def _to_bc_h_w_2(x, x_shape):
    """(b, h, w, 2c) -> (b*c, h, w, 2)"""
    x = tf.transpose(x, [0, 3, 1, 2])
    x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2]), 2))
    return x

  @staticmethod
  def _to_bc_h_w(x, x_shape):
    """(b, h, w, c) -> (b*c, h, w)"""
    x = tf.transpose(x, [0, 3, 1, 2])
    x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2])))
    return x

  @staticmethod
  def _to_b_h_w_c(x, x_shape):
    """(b*c, h, w) -> (b, h, w, c)"""
    x = tf.reshape(
        x, (-1, int(x_shape[3]), int(x_shape[1]), int(x_shape[2]))
    )
    x = tf.transpose(x, [0, 2, 3, 1])
    return x

In [ ]:
######################################backbone部分######################################
def feature_extraction_CNN(input_shape, training_parameter= False,n_filters=64):
  # X_input=L.Input(input_shape)
  conv1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(input_shape)
  
  conv1 = BN_LeakyReLU(conv1)
  conv1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(conv1)
  conv1 = BN_LeakyReLU(conv1)
  
  pool1 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv1)
  # print(pool1)
  #可变形block A1
  # print(pool1)
  offset_conv2_1 = ConvOffset2D(2*n_filters)(pool1)
  
  conv2_1 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_1)
  conv2_1 = BN_LeakyReLU(conv2_1)
  
  offset_conv2_2 = ConvOffset2D(n_filters)(conv2_1)
  conv2_2 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_2)
  conv2_2 = BN_LeakyReLU(conv2_2)
  
  offset_conv2_3 = ConvOffset2D(int(0.5*n_filters))(conv2_2)
  conv2_3 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_3)
  conv2_3 = BN_LeakyReLU(conv2_3)
  
  conv2_4 = L.concatenate([conv2_1, conv2_2, conv2_3], axis=-1)
  # print(conv2_4)
  conv2_5 = L.add([conv2_4, pool1])

  #可变形block A2
  offset_conv2_6 = ConvOffset2D(2*n_filters)(conv2_5)
  conv2_6 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_6)
  conv2_6 = BN_LeakyReLU(conv2_6)
  
  offset_conv2_7 = ConvOffset2D(n_filters)(conv2_6)
  conv2_7 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_7)
  conv2_7 = BN_LeakyReLU(conv2_7)
  
  offset_conv2_8 = ConvOffset2D(int(0.5*n_filters))(conv2_7)
  conv2_8 = L.Conv2D(int(0.5*n_filters), (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv2_8)
  conv2_8 = BN_LeakyReLU(conv2_8)
  
  conv2_9 = L.concatenate([conv2_6, conv2_7, conv2_8], axis=-1)
  
  conv2_10 = L.add([conv2_9, conv2_5])
  
  
  #pool2 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv2_5)
  
  conv3 = L.Conv2D(4*n_filters, (3, 3), padding='valid', strides=(2, 2), kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(conv2_5)
  
  offset_conv3_1 = ConvOffset2D(4*n_filters)(conv3)
  
  conv3_1 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_1)
  conv3_1 = BN_LeakyReLU(conv3_1)
  
  offset_conv3_2 = ConvOffset2D(2*n_filters)(conv3_1)
  conv3_2 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_2)
  conv3_2 = BN_LeakyReLU(conv3_2)
  offset_conv3_3 = ConvOffset2D(n_filters)(conv3_2)
  conv3_3 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_3)
  conv3_3 = BN_LeakyReLU(conv3_3)
  
  conv3_4 = L.concatenate([conv3_1, conv3_2, conv3_3], axis=-1)
  
  conv3_5 = L.add([conv3_4, conv3])

  #改了一下4*
  offset_conv3_6 = ConvOffset2D(n_filters)(conv3_2)
  
  conv3_6 = L.Conv2D(2*n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_6)
  conv3_6 = BN_LeakyReLU(conv3_6)
  
  offset_conv3_7 = ConvOffset2D(2*n_filters)(conv3_6)
  conv3_7 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_7)
  conv3_7 = BN_LeakyReLU(conv3_7)
  
  offset_conv3_8 = ConvOffset2D(n_filters)(conv3_7)
  conv3_8 = L.Conv2D(n_filters, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(offset_conv3_8)
  conv3_8 = BN_LeakyReLU(conv3_8)
  
  conv3_9 = L.concatenate([conv3_6, conv3_7, conv3_8], axis=-1)
  
  conv3_10 = L.add([conv3_9, conv3_5])
  
  
  #pool3 = L.MaxPool2D(pool_size=(2, 2),padding='same')(conv3)
  #输出128维计算度量
  # print(conv3_10)
  conv4 = L.Conv2D(n_filters*2, (3, 3), padding='same', kernel_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.0001))(conv3_10)
  # print(conv4)
  gap = L.GlobalAvgPool2D()(conv4)
  # print(gap)
  #增加功能
  X = Dense(1024,kernel_regularizer=regularizers.l2(0.01),name='dense_layer1')(gap)
  X1 = Dropout(rate=0.5)(X,training=training_parameter)#打开training模式，测试时也使用dropout
  X2 = Dense(256,kernel_regularizer=regularizers.l2(0.01),name='dense_layer2')(X1)
  X3 = Dropout(rate=0.5)(X2,training=training_parameter)
  spatial_result = X3
  print('输出向量维度：' + str(spatial_result.shape))
  return spatial_result

#预测

In [ ]:
def read_image(filename):
  img = tiff.imread(filename)
  # img = cv2.imread(filename,-1)
  # img = np.asarray(img, dtype=np.float32)
  return img

In [ ]:
def Patch(data, height_index, width_index):
  
  height_slice = slice(height_index-r, height_index + r+1)
  width_slice = slice(width_index-r, width_index + r+1)
  patch = data[height_slice, width_slice, :]
  
  return patch

In [ ]:
def sample_normalization(data):
    """
    normalize each sample to 0-1
    Input:
        sample
    Output:
        Normalized sample
    """
    if np.max(data) == np.min(data):
        return np.ones_like(data, dtype=np.float32) * 1e-6
    else:
        return 1.0 * (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
#判断是否背景值
def judge_underground(img,height,width):
  # print(Counter(img[height,width,:]))
  if Counter(img[height,width,:])[0] == 12:
    return True
  else:
    return False

In [ ]:
def get_patch(data,height_index,width_index,r=7):
  patch = data[height_index-r:height_index+r+1,width_index-r:width_index+r+1,:]
  return patch

#大图切片预测

In [ ]:
datapath = '/content/gdrive/MyDrive/CM/img/zhangbei_subset.tif'
r=7
hchn = 12
PATCH_SIZE = 15
step = 1  # 每隔几个像素预测一次，如果每个像素都预测，则 step 为 1
NUM_CLASS = 7

In [ ]:
def predict_little(img,mean,model):
  height, width, channel = img.shape
  # img=np.pad(img,((r,r),(r,r),(0,0)),'symmetric')
  outputs = np.zeros((height, width))
  # print('\n输入patch大小为:',img.shape)
  # print('输出patch大小为：',outputs[r:height-r,r:width-r].shape)
  for i in range(r, height-r, step):
    for j in range(r, width-r, step):
      if judge_underground(img,i,j) == False:
        # print(img[i,j,:])
        image_patch = Patch(img, i, j)  # 切 patch
        #巨大BUG！忘了已经归一化过了
        # image_patch = sample_normalization(image_patch)
        image_patch = image_patch - mean
        X_test_image = image_patch[np.newaxis,...]
        
        # count += 1
        # # if count%1000 == 0:
        # #     print('step={}'.format(count))
        pred = (np.argmax(model.predict(X_test_image)))
        # #pred_cls = np.argmax(pred, axis=1)
        pred_num = np.asarray(pred, dtype=np.int8)
        outputs[i,j] = pred_num
        
        # # outputs[i+int(PATCH_SIZE/2)][j+int(PATCH_SIZE/2)] = pred_num  # 原始代码
        # outputs[i//step][j//step] = pred_num
      else:
        outputs[i,j] = 0
  return outputs[r:height-r,r:width-r]
def splitpredict(path,model):
  img = read_image(path)
  height, width, channel = img.shape
  img = sample_normalization(img)
  mean_value = np.mean(img)
  h_step = img.shape[0] // 256
  w_step = img.shape[1] // 256
  h_last = img.shape[0] % 256
  w_last = img.shape[1] % 256
  # outputs = np.zeros((height, width))
  #循环切图
  # plt.figure()
  for h in tqdm(range(17,18),position=0):
    for w in tqdm(range(24,25),position=0):
      if (w < w_step - 1 and w > 0) and ( h > 0 and h < h_step -1):#内部
        image_sample = img[(h * 256 - r):(h * 256 + 256 + r),(w * 256 - r):(w * 256 + 256 + r), :]
        # print('内部',image_sample.shape)

      elif (w==0) and (h==0): #左上边界
        image_sample = img[0:256+r,0:256+r, :]
        image_sample = np.pad(image_sample,((r,0),(r,0),(0,0)),'symmetric')
        # print('左上',image_sample.shape)

      elif ( w==0 ) and (h < h_step -1):#左边界
        image_sample = img[(h * 256 - r):(h * 256 + 256 + r),0:256+r, :]
        image_sample = np.pad(image_sample,((0,0),(r,0),(0,0)),'symmetric')
        # print('左边界',image_sample.shape)

      elif (w < w_step -1) and (h == 0 ): #上边界
        image_sample = img[0:256+r,(w * 256 - r):(w * 256 + 256 + r), :]
        image_sample = np.pad(image_sample,((r,0),(0,0),(0,0)),'symmetric')
        # print('上边界',image_sample.shape)

      elif ( w == w_step - 1) and (h ==0):#右上边界
        image_sample = img[0:256+r,(w * 256 - r):(w * 256 + w_last), :]
        image_sample = np.pad(image_sample,((r,0),(0,0),(0,0)),'symmetric')
        # print('右上边界',image_sample.shape)

      elif (w == w_step -1) and (h < h_step -1):#右边界
        image_sample = img[(h * 256 - r):(h * 256 + 256 + r),(w * 256 - r):(w * 256 + w_last), :]
        # print('右边界',image_sample.shape)

      elif (w < w_step - 1 and w > 0) and (h == h_step -1):#下边界
        image_sample = img[(h * 256 - r):(h * 256 + h_last),(w * 256 - r):(w * 256 + 256 + r), :]
        # print('下边界',image_sample.shape)

      elif (w == 0) and (h == h_step -1):#左下边界
        image_sample = img[(h * 256 - r):(h * 256 + h_last),0:(w * 256 + 256 + r), :]
        image_sample = np.pad(image_sample,((0,0),(r,0),(0,0)),'symmetric')
        # print('左下边界',image_sample.shape)

      else:#右下边界
        image_sample = img[(h * 256 - r):(h * 256 + h_last),(w * 256 - r):(w * 256 + w_last), :]
        # print('右下边界',image_sample.shape)

      pred_little = predict_little(image_sample,mean_value,model)
      np.save('/content/gdrive/MyDrive/CM/RSL/paperimage_cut/cut_'+str(h)+'_'+str(w), pred_little)


In [ ]:
model1 = import_model('/content/gdrive/MyDrive/CM/RSL/CE_highsoftmx_highmix_lowups4.h5',train_para = False)

输出向量维度：(None, 256)


In [ ]:
splitpredict(datapath,model1)

100%|██████████| 1/1 [32:38<00:00, 1958.12s/it]


#拼接保存图


In [ ]:
cv2.imwrite("/content/gdrive/MyDrive/CM/RSL/paperimage/zhangbei.png", predict_image)

In [ ]:
os.chdir('/content/gdrive/MyDrive/CM/RSL/paperimage_cut')

In [ ]:
#拼接图片
for i in range(31):
  result  = np.load('cut_'+str(i)+'_0.npy')
  for j in tqdm(range(1,43),position=0):
    filepath = 'cut_'+str(i)+'_'+str(j)+'.npy'
    patch = np.load(filepath)
    result = np.concatenate([result, patch], axis=1)
  np.save('cut_'+str(i)+'.npy', result)
    

In [ ]:
result  = np.load('cut_0.npy')
for i in range(1,31):
  filepath = 'cut_'+str(i)+'.npy'
  patch = np.load(filepath)
  result = np.concatenate([result, patch], axis=0)
np.save('zhangbei_map.npy', result)